# Term Frequency–Inverse Document Frequency (TF-IDF) - Experimento

Convert a collection of raw documents to a matrix of TF-IDF features.
Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declare parâmetros e hiperparâmetros para o modelo
Os componentes podem declarar (e usar) estes parâmetros como padrão:
- dataset
- target

Use estes parâmetros para carregar/salvar conjutos de dados, modelos, métricas e figuras com a ajuda do [SDK da PlatIAgro](https://platiagro.github.io/sdk/). <br>
É possível também declarar parâmetros personalizados para serem definidos ao executar um experimento. 

Selecione os hiperparâmetros e seus respectivos valores para serem usados ao treinar o modelo:
- language

Estes parâmetros são alguns dos oferecidos pela classe do modelo, você também pode utilizar outros existentes. <br>
Dê uma olhada nos [parâmetros do modelo](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer) para mais informações.

In [158]:
# parâmetros
dataset = "imdb.csv" #@param {type:"string"}
target = "label" #@param {type:"feature", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
text = "text" #@param {type:"string", label:"Texto alvo", description:"Nome da coluna do texto alvo pertencente ao dataset."}
language = "english" #@param ["portuguese", "english"] {type:"string", label:"Linguagem", description:"Linguagem da qual os stopwords pertencem. Deve ser a mesma utilizada no dataset."}

# selected features to perform the model
filter_type = "incluir" #@param ["incluir","remover"] {type:"string",multiple:false,label:"Modo de seleção das features",description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção 'incluir'. Caso deseje informar as features que não devem ser utilizadas, selecione 'remover'. "}
model_features = "text" #@param {type:"feature",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

# features to apply One Hot Encoder
one_hot_features = "" #@param {type:"feature",multiple:true,label:"Features para fazer codificação one-hot", description: "Seu modelo utilizará a codificação one-hot para as features selecionadas. As demais features categóricas serão codificadas utilizando a codificação ordinal."}


## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [159]:
import pandas as pd

df = pd.read_csv(f'/tmp/data/{dataset}')
df

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
995,negative,There are many different versions of this one ...,True
996,positive,Once upon a time Hollywood produced live-actio...,True
997,negative,Wenders was great with Million $ Hotel.I don't...,True
998,negative,Although a film with Bruce Willis is always wo...,True


## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [160]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [161]:
df.dropna(subset = [target],inplace=True)
y = df[target].to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [162]:
if filter_type == 'incluir':
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        featuretypes = featuretypes[columns_index]
    else:
        columns_to_filter = columns
else:
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns,columns_index)
        featuretypes = np.delete(featuretypes,columns_index)
    else:
        columns_to_filter = columns

# keep the features selected
df_model = df[columns_to_filter]
X = df_model.to_numpy()

## Divisão do datset em subconjuntos de treino e teste

Subconjunto de Treino: amostras de dados usado para treinar o modelo (``fit``). <br>
Subconjunto de Teste: a amostra de dados usada para fornecer uma avaliação imparcial de um modelo adequado ao conjunto de dados de treinamento.

In [163]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

In [164]:
mapeamento = {'positive': True, 'negative': False}
y_train = [mapeamento.get(i, i) for i in y_train]
y_test = [mapeamento.get(i, i) for i in y_test]

## Busca por stopwords

Stopwords (ou palavras de parada) são palavras que geralmente se referem às mais comuns em um idioma ou em um corpus. <br>
Elas podem ser ignoradas com segurança sem sacrificar o significado da frase, pois são palarvas que não agregram muito significado.

In [165]:
!pip install nltk --quiet
import nltk

# Download stopwords from nltk
nltk.download('stopwords')

# Get a list of stopwords for the defined language
stopwords = nltk.corpus.stopwords.words(language)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Processamento do texto

Funções auxiliares para processamento dos dados.

In [166]:

from re import sub
from collections import defaultdict
from functools import reduce


def tokenize_without_punctuation(text_list: list = None):
    """Tokenize without ponctuation.

    Args:
        text_list (list): a list of texts to be used.

    Returns:
        A list of tokenized text without punctuation.
    """
    tokenize_list = list()
    punctuation_pattern = "[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]"
    html_tag_pattern = "<.*?>"

    for text in text_list:
        text = sub(html_tag_pattern, ' ', text[0])
        tokenize_list.append(sub(punctuation_pattern, ' ', text).split(' '))

    return tokenize_list


def top_tokens_stopwords(token_list: list, percentage: float = 0.01):
    """Selects the most relevant stops words of the tokerized texts.

    Args:
        token_list (list): list of tokens.
        percentage (float): percentage threshold.
    """
    vocabulary = defaultdict(int)

    for sample in token_list:
        for token in sample:
            vocabulary[token] += 1

    all_tokens = sorted(vocabulary.items(), key=lambda token: token[1], reverse=True)
    top_tokens = all_tokens[:int(len(all_tokens) * percentage)]

    return [token[0] for token in top_tokens]


def remove_specific_tokens(token_list: list, tokens_to_be_removed: list = None):
    """Removes specific tokens from a token list.

    Args:
        token_list (list): list of tokens from which other tokens will be removed.
        tokens_to_be_removed (list): list of tokens that need to be removed.
    """
    token_list_ = list()

    if tokens_to_be_removed is None:
        tokens_to_be_removed = top_tokens_stopwords(token_list)

    for sample in token_list:
        sample = list(set(sample) - set(tokens_to_be_removed))
        token_list_.append(sample)

    return token_list_


def token_restructuring(token_list: list):
    """Reduce a nested list of tokens to a single list (1D).
    
    Args:
        token_list (list): list to be work on.
    """
    return reduce(lambda x, y: x + y, token_list)

## Criação da matriz de TF-IDF

In [167]:
vocab = tokenize_without_punctuation(X_train)
top_tokens = top_tokens_stopwords(vocab)
vocab = remove_specific_tokens(vocab,top_tokens)
vocab = remove_specific_tokens(vocab,stopwords)
text = [' '.join(tokens) for tokens in vocab]
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
X_train_tfidf = vectorizer.transform(text).toarray()
X_train_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [168]:
X_test_good_format = token_restructuring([list(row) for row in X_test])
vectorizer = TfidfVectorizer()
vectorizer.fit(X_test_good_format)
X_test_tfidf = vectorizer.transform(X_test_good_format).toarray()
X_test_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Criaçãod do Dataloader

In [169]:
from torch.utils.data import Dataset, DataLoader
from numpy import genfromtxt
import torch

class ImdbDataset(Dataset):
    def __init__(self, X, target):
        super(ImdbDataset, self).__init__()

        self.x = torch.tensor(X).type(torch.FloatTensor)
        self.target = torch.tensor(target).type(torch.LongTensor)


    def __len__(self):
    
        return len(self.x)
  
    def __getitem__(self, index):
        return self.x[index], self.target[index] 
    

Criando e Testando Dataloaders

In [173]:
torch_ds_train = ImdbDataset(X_train_tfidf,y_train)
torch_ds_test = ImdbDataset(X_test_tfidf,y_test)
print("------->Testando Dataloader de treino<-------")
x1, y1 = torch_ds_train[0]
print(x1)
print(y1)
print("------->Testando Dataloader de teste<-------")
x2, y2 = torch_ds_test[0]
print(x2)
print(y2)

------->Testando Dataloader de treino<-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor(1)
------->Testando Dataloader de teste<-------
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor(0)


## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [174]:
from platiagro import save_model

save_model(torch_ds_train=torch_ds_train,
           torch_ds_test=torch_ds_test)

/opt/conda/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
